### **Import packages, download datasets and weights**

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
! git clone https://github.com/smitasasindran/MidasYoloPlanercnn.git

Cloning into 'MidasYoloPlanercnn'...
remote: Enumerating objects: 145, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 145 (delta 34), reused 143 (delta 32), pack-reused 0
Receiving objects: 100% (145/145), 11.37 MiB | 23.34 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [3]:
%cd MidasYoloPlanercnn
# ! ls

/content/MidasYoloPlanercnn


In [4]:
!ls

cfg   dataloader  testing_modules.py  train.py	  utils
data  models	  test.py	      transforms  weights


In [5]:
# Copy weights
!mkdir weights
!cp /content/gdrive/MyDrive/eva5/ass15/model-f6b98070.pt weights/
!cp /content/gdrive/'My Drive'/eva5/ass13/yolov3-spp-ultralytics.pt weights/
!ls weights

mkdir: cannot create directory ‘weights’: File exists
model-f6b98070.pt  tempfile  yolov3-spp-ultralytics.pt


In [6]:
!mkdir /content/rawdata
%cd /content/rawdata
!ls

/content/rawdata


In [7]:
# # !cp /content/gdrive/'My Drive'/eva5/ass15a/planes/planes.zip rawdata/
!cp /content/gdrive/'My Drive'/eva5/ass15a/depth/depth.zip .
!cp /content/gdrive/'My Drive'/eva5/ass15a/images/images.zip . 
!cp /content/gdrive/'My Drive'/eva5/ass15a/labels/labels.zip .

! ls

depth.zip  images.zip  labels.zip


In [10]:
!unzip images.zip -d images
!unzip labels.zip #-d labels
!unzip depth.zip -d depth

# !unzip planes.zip -d planes
!ls


unzip:  cannot find or open images.zip, images.zip.zip or images.zip.ZIP.
unzip:  cannot find or open labels.zip, labels.zip.zip or labels.zip.ZIP.
unzip:  cannot find or open depth.zip, depth.zip.zip or depth.zip.ZIP.
cfg   dataloader  testing_modules.py  train.py	  utils
data  models	  test.py	      transforms  weights


In [12]:
%cd ../MidasYoloPlanercnn
!ls
!pwd

cfg   dataloader  testing_modules.py  train.py	  utils
data  models	  test.py	      transforms  weights
/content/MidasYoloPlanercnn


In [14]:
# Copy dataset
# Delete existing images and labels, and copy new ones from own gdrive folder
!rm data/customdata/images/*
!rm data/customdata/labels/*
!rm data/customdata/midas/*
# !rm data/customdata/planes/*

!mv /content/rawdata/images/* data/customdata/images/
!mv /content/rawdata/labels/* data/customdata/labels/
!mv /content/rawdata/depth/* data/customdata/midas/
# !mv /content/rawdata/planes/* data/customdata/planes/


rm: cannot remove 'data/customdata/images/*': No such file or directory
rm: cannot remove 'data/customdata/labels/*': No such file or directory
rm: cannot remove 'data/customdata/midas/*': No such file or directory


In [15]:
# Copy custom config files
!cp ../gdrive/'My Drive'/eva5/ass13/custom/custom.names data/customdata/
!cp ../gdrive/'My Drive'/eva5/ass13/custom/custom.data data/customdata/
!cp ../gdrive/'My Drive'/eva5/ass13/custom/custom_train.txt data/customdata/
!cp ../gdrive/'My Drive'/eva5/ass13/custom/custom_test.txt data/customdata/
!cp ../gdrive/'My Drive'/eva5/ass13/custom/train.shapes data/customdata/
!cp ../gdrive/'My Drive'/eva5/ass13/custom/test.shapes data/customdata/


In [16]:
# These files have a problem, label coordinate > 1. Manually fix or else delete label:

!rm data/customdata/labels/8c7a3576.txt
!rm data/customdata/labels/8c7b2016.txt
!rm data/customdata/labels/8c7d69b6.txt

In [17]:
# Imports
import torch
import torch.nn as nn
from torchvision.transforms import Compose
from torchsummary import summary

import cv2
import numpy as np
import math
import glob
import os
import sys
import re



### **Training Midas and Yolo combined model**

In [20]:

!python train.py --data data/customdata/custom.data --batch 16 --cache --cfg cfg/yolov3-custom.cfg --epochs 100 --load-init --img-size 64 --nosave



Namespace(accumulate=4, adam=False, batch_size=16, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=100, evolve=False, freeze_encoder=False, img_size=[64], initWeights=True, init_midas_weights='weights/model-f6b98070.pt', init_planercnn_weights='', init_yolo_weights='weights/yolov3-spp-ultralytics.pt', load_init=True, multi_scale=False, name='', nosave=True, notest=False, rect=False, resume=False, single_cls=False, weights='weights/yolov3-spp-ultralytics.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

2020-12-06 07:27:03.417470: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
Loading weights:  model-f6b98070.pt
Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master
Downloading: "https://download.pytorch.org/models/ig

In [22]:
# Died at 50, but don't have resume. So running for 50 more from saved best weights
!python train.py --data data/customdata/custom.data --batch 16 --cache --cfg cfg/yolov3-custom.cfg --epochs 50 --weights weights/best.pt --img-size 64 --nosave


Namespace(accumulate=4, adam=False, batch_size=16, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=50, evolve=False, freeze_encoder=False, img_size=[64], initWeights='', init_midas_weights='weights/model-f6b98070.pt', init_planercnn_weights='', init_yolo_weights='weights/yolov3-spp-ultralytics.pt', load_init=False, multi_scale=False, name='', nosave=True, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

2020-12-06 08:50:40.289226: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
Loading weights:  model-f6b98070.pt
Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master
Adding to routes from convolutional: 4
Adding to routes from convolution

In [24]:
! cp weights/best.pt /content/gdrive/'My Drive'/eva5/ass13/custom/weights/best15_a.pt
! cp weights/last.pt /content/gdrive/'My Drive'/eva5/ass13/custom/weights/last15_a.pt

In [26]:
!python train.py --data data/customdata/custom.data --batch 16 --cache --cfg cfg/yolov3-custom.cfg --epochs 100 --weights weights/best.pt --img-size 128 --nosave



Namespace(accumulate=4, adam=False, batch_size=16, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=100, evolve=False, freeze_encoder=False, img_size=[128], initWeights='', init_midas_weights='weights/model-f6b98070.pt', init_planercnn_weights='', init_yolo_weights='weights/yolov3-spp-ultralytics.pt', load_init=False, multi_scale=False, name='', nosave=True, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

2020-12-06 10:53:28.969748: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
Loading weights:  model-f6b98070.pt
Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master
Adding to routes from convolutional: 4
Adding to routes from convoluti

In [27]:
!python train.py --data data/customdata/custom.data --batch 8 --cache --cfg cfg/yolov3-custom.cfg --epochs 30 --weights weights/best.pt --img-size 256 --nosave


Namespace(accumulate=4, adam=False, batch_size=8, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=30, evolve=False, freeze_encoder=False, img_size=[256], initWeights='', init_midas_weights='weights/model-f6b98070.pt', init_planercnn_weights='', init_yolo_weights='weights/yolov3-spp-ultralytics.pt', load_init=False, multi_scale=False, name='', nosave=True, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

2020-12-06 12:14:12.196652: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
Loading weights:  model-f6b98070.pt
Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master
Adding to routes from convolutional: 4
Adding to routes from convolution

In [33]:
# Unfreeze encoder...
!python train.py --data data/customdata/custom.data --batch 4 --cache --cfg cfg/yolov3-custom.cfg --epochs 10 --weights weights/best-256.pt --img-size 512 --nosave


Namespace(accumulate=4, adam=False, batch_size=4, bucket='', cache_images=True, cfg='cfg/yolov3-custom.cfg', data='data/customdata/custom.data', device='', epochs=10, evolve=False, freeze_encoder=False, img_size=[512], initWeights='', init_midas_weights='weights/model-f6b98070.pt', init_planercnn_weights='', init_yolo_weights='weights/yolov3-spp-ultralytics.pt', load_init=False, multi_scale=False, name='', nosave=True, notest=False, rect=False, resume=False, single_cls=False, weights='weights/best-256.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla T4', total_memory=15079MB)

2020-12-06 14:38:45.366069: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Run 'tensorboard --logdir=runs' to view tensorboard at http://localhost:6006/
Loading weights:  model-f6b98070.pt
Using cache found in /root/.cache/torch/hub/facebookresearch_WSL-Images_master
Adding to routes from convolutional: 4
Adding to routes from convolu